In [ ]:
!pip3 install -q transformers
!pip3 install emoji==0.6.0
# !pip3 install fastopic

In [ ]:
!pip3 install bertopic

In [ ]:
import os
import requests
import pandas as pd
from transformers import pipeline
from IPython.display import display, Markdown

import openai
from google.colab import userdata

In [ ]:
openai.api_key = userdata.get('open')
token = userdata.get('fb_api')

In [ ]:
post_id = ""  # Example post ID #input your post id
graph_api_version = "v16.0"

url = f"https://graph.facebook.com/{graph_api_version}/{post_id}/comments"

# IMPORTANT: Notice how we add `comments{message,from,created_time}` to the fields
params = {
    "access_token": token,
    "fields": "message,from,created_time,comments{message,from,created_time}",
    "limit": 50
}

response = requests.get(url, params=params)
data = response.json()

if "error" in data:
    print(f"Error from Graph API: {data['error']}")
else:
    # Top-level comments
    top_level_comments = data.get("data", [])

    # Let's store all top-level comments plus replies in a list
    all_comments_data = []

    # Gather pages of top-level comments (pagination)
    next_page = data.get("paging", {}).get("next")

    while next_page:
        next_resp = requests.get(next_page)
        next_data = next_resp.json()
        new_top_level = next_data.get("data", [])
        top_level_comments.extend(new_top_level)
        next_page = next_data.get("paging", {}).get("next")

    # Process each top-level comment and potential replies
    for comment in top_level_comments:
        # Basic fields
        comment_id = comment.get("id")
        message = comment.get("message")
        from_user = comment.get("from", {}).get("name")  # or "id"
        created_time = comment.get("created_time")

        # Add the top-level comment to our consolidated list
        all_comments_data.append({
            "comment_id": comment_id,
            "parent": None,  # top-level has no parent
            "message": message,
            "from_user": from_user,
            "created_time": created_time
        })

        # Now check if there are replies (nested `comments` object)
        nested_comments = comment.get("comments", {}).get("data", [])
        for reply in nested_comments:
            reply_id = reply.get("id")
            reply_msg = reply.get("message")
            reply_user = reply.get("from", {}).get("name")
            reply_time = reply.get("created_time")

            # Each reply references the parent (the top-level comment ID)
            all_comments_data.append({
                "comment_id": reply_id,
                "parent": comment_id,
                "message": reply_msg,
                "from_user": reply_user,
                "created_time": reply_time
            })

    # Convert it all into a DataFrame for easy viewing
    df_all = pd.DataFrame(all_comments_data)
    print("Top-level comments and replies:")
    display(df_all)


Top-level comments and replies:


,comment_id,parent,message,from_user,created_time
0,122167301912292583_1290925778870832,None,I love my Dotti patch I got from my MD.,None,2025-02-27T02:41:56+0000
1,122167301912292583_885288293626347,122167301912292583_1290925778870832,Elizabeth Houser absolute game changer,None,2025-03-06T14:01:07+0000
2,122167301912292583_658925796699916,122167301912292583_1290925778870832,Elizabeth Houser I have been HOT my entire lif...,None,2025-03-06T14:02:24+0000
3,122167301912292583_1722538385273405,122167301912292583_1290925778870832,Elizabeth Houser me too! I live in the south s...,None,2025-03-09T17:08:07+0000
4,122167301912292583_548120231031498,122167301912292583_1290925778870832,Elizabeth Houser what’s a Dotti patch? My gyno...,None,2025-03-11T10:50:14+0000
...,...,...,...,...,...
505,122167301912292583_1153142709836300,None,"My Dr says ,increased risk of high blood press...",None,2025-03-16T11:31:07+0000
506,122167301912292583_1333655327760105,None,HRT causes cancer,None,2025-03-15T13:06:44+0000
507,122167301912292583_682818524170123,None,"I won’t use anything, but biodenticals. synth...",None,2025-03-12T16:44:10+0000
508,122167301912292583_674450998465798,None,Are they bioidentical?,None,2025-03-12T16:42:05+0000


In [ ]:
list_comments =df_all[['message', 'created_time']].to_dict('records')

In [ ]:
messages = df_all['message'].tolist()

In [ ]:
model = pipeline("text-classification", model="MarieAngeA13/Sentiment-Analysis-BERT")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/944 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
sentiments_allcomments = model(messages)

In [ ]:
combined = []
for comment, result in zip(list_comments, sentiments_allcomments):
  # print(comment, result)
  combined.append({
        "created": comment['created_time'],
        "comment": comment['message'],
        "label": result["label"],
        "score": result["score"],
    })

In [ ]:
df = pd.DataFrame(combined)
# df.to_csv('comments_sentiment_2.csv', index=False)
df.head()

,created,comment,label,score
0,2025-02-27T02:41:56+0000,I love my Dotti patch I got from my MD.,positive,0.969882
1,2025-03-06T14:01:07+0000,Elizabeth Houser absolute game changer,neutral,0.908427
2,2025-03-06T14:02:24+0000,Elizabeth Houser I have been HOT my entire lif...,negative,0.896481
3,2025-03-09T17:08:07+0000,Elizabeth Houser me too! I live in the south s...,neutral,0.857772
4,2025-03-11T10:50:14+0000,Elizabeth Houser what’s a Dotti patch? My gyno...,neutral,0.930616


In [ ]:
def calculate_scoresbydate(df):
    # 1) Convert the 'created' column to datetime objects
    df['datetime'] = pd.to_datetime(df['created'], format='%Y-%m-%dT%H:%M:%S%z')

    # 2) Extract just the date portion
    df['date'] = df['datetime'].dt.date

    # 3) Group by date and compute (sum of positive) - (sum of negative)
    df_scores_by_date = (
        df.groupby('date')
          .apply(lambda g: (
              g.loc[g['label'] == 'positive', 'score'].sum()
              - g.loc[g['label'] == 'negative', 'score'].sum()
          ))
          .reset_index(name='total_effect')
    )

    # 4) Return the new DataFrame with date and total_effect columns
    return df_scores_by_date

In [ ]:
calculate_scoresbydate(df)

In [ ]:
prompt_text = f"""
Please summarize the following text. Focus on the key points, main arguments, and any important statistics or insights.

Text:
{messages}
Summary:
"""

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",  # or "gpt-4" if you have access
    messages=[
        {
            "role": "system",
            "content": """The comments are posted on a FB post. Can you highlight the negative and positive and summarise it. """
        },
        {
            "role": "user",
            "content": prompt_text
        }
    ],

    temperature=0.2
)

In [ ]:
# print(response.choices[0].message.content)
display(Markdown(response.choices[0].message.content))

Positive:
- Many users shared their positive experiences with using the Dotti patch and hormone replacement therapy (HRT), mentioning improvements in symptoms like hot flashes, mood swings, and weight gain.
- Users appreciated the ease of use and effectiveness of the patch, with some mentioning that it was a game-changer for them.
- Some users mentioned feeling like themselves again, having relief from symptoms, and experiencing weight loss and improved quality of life after starting HRT.

Negative:
- Some users expressed concerns about the cost of HRT, side effects like weight gain, and challenges with finding the right dosage or delivery method.
- There were mentions of doctors being hesitant to prescribe HRT due to concerns about cancer risks, blood clots, and other health issues.
- A few users shared negative experiences with HRT, mentioning side effects like yeast infections, breast lumps, and worsening symptoms.

Overall, the comments highlighted a mix of positive experiences with the Dotti patch and HRT, as well as concerns and challenges related to cost, side effects, and medical advice.